In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter



from programs.utils import levenshtein

In [3]:
lvl_3 = pd.read_csv('datafiles/analyses/name_variants/matches_links_exact_lvl_3.csv', sep="|", index_col=False)
lvl_3.columns = ['edit_distance', 'id_1', 'target_name_1', 'target_name_2', 'target_name_3', 'target_name_4', 'id_2', 'candidate_name_1', 'candidate_name_2', 'candidate_name_3', 'candidate_name_4']
lvl_4 = pd.read_csv('datafiles/analyses/name_variants/matches_links_exact_lvl_4.csv', sep='|', index_col=False)
lvl_4.columns = ['edit_distance', 'id_1', 'target_name_1', 'target_name_2', 'target_name_3', 'target_name_4', 'id_2', 'candidate_name_1', 'candidate_name_2', 'candidate_name_3', 'candidate_name_4']
lvl_4 = lvl_4.fillna(" ")
lvl_5 = pd.read_csv('datafiles/analyses/name_variants/matches_links_exact_lvl_5.csv', sep='|', index_col=False)
lvl_5.columns = ['edit_distance', 'id_1', 'target_name_1', 'target_name_2', 'target_name_3', 'target_name_4', 'id_2', 'candidate_name_1', 'candidate_name_2', 'candidate_name_3', 'candidate_name_4']
lvl_5 = lvl_5.fillna(" ")

In [4]:
first_names_counts = Counter()
fam_names_counts = Counter()

for index, row in lvl_3.iterrows():
    for i in [1,3]:
        name1 = row['target_name_' + str(i)]
        name2 = row['candidate_name_' + str(i)]
        
        # naam varianten moeten niet gesplits worden naar voorkomen
        # dus naam1, naam2 is hetzelfde als naam2,naam1
        if first_names_counts[(name1, name2)]:
            first_names_counts[(name1, name2)] += 1
        else:
            first_names_counts[(name2, name1)] += 1
    for i in [2,4]:
        name1 = row['target_name_' + str(i)]
        name2 = row['candidate_name_' + str(i)]
        if fam_names_counts[(name1, name2)]:
            fam_names_counts[(name1, name2)] += 1
        else:
            fam_names_counts[(name2, name1)] += 1

In [5]:
print first_names_counts
print fam_names_counts

Counter({('maria', 'maria'): 26220, ('pieter', 'pieter'): 23426, ('jan', 'jan'): 22886, ('cornelis', 'cornelis'): 17086, ('johanna', 'johanna'): 15309, ('cornelia', 'cornelia'): 14782, ('adriana', 'adriana'): 12175, ('jacobus', 'jacobus'): 11987, ('adriaan', 'adriaan'): 11936, ('pieternella', 'pieternella'): 11935, ('marinus', 'marinus'): 10218, ('willem', 'willem'): 9728, ('johannes', 'johannes'): 9194, ('maatje', 'maatje'): 8014, ('janna', 'janna'): 7844, ('abraham', 'abraham'): 7621, ('jacob', 'jacob'): 7208, ('anna', 'anna'): 7127, ('neeltje', 'neeltje'): 6708, ('catharina', 'catharina'): 6210, ('jacoba', 'jacoba'): 5637, ('jannetje', 'jannetje'): 5267, ('leendert', 'leendert'): 4254, ('sara', 'sara'): 4148, ('elizabeth', 'elizabeth'): 4126, ('hendrik', 'hendrik'): 4003, ('elisabeth', 'elisabeth'): 3660, ('petrus', 'petrus'): 3592, ('jacomina', 'jacomina'): 3448, ('willemina', 'willemina'): 3444, ('johannis', 'johannis'): 2974, ('dina', 'dina'): 2974, ('johannes', 'johannis'): 2756

In [6]:
lvl_3_first_name_variants = pd.DataFrame()
for (name1, name2) in first_names_counts:
    lvl_3_first_name_variants = lvl_3_first_name_variants.append(pd.Series([name1, name2, first_names_counts[(name1, name2)]]), ignore_index=True)

In [7]:
lvl_3_first_name_variants.columns = ["name1", "name2", "count"]
print lvl_3_first_name_variants.sort_values("count", ascending=False)

            name1        name2    count
4056        maria        maria  26220.0
5082       pieter       pieter  23426.0
4427          jan          jan  22886.0
4464     cornelis     cornelis  17086.0
691       johanna      johanna  15309.0
4942     cornelia     cornelia  14782.0
206       adriana      adriana  12175.0
478       jacobus      jacobus  11987.0
401       adriaan      adriaan  11936.0
201   pieternella  pieternella  11935.0
3336      marinus      marinus  10218.0
5680       willem       willem   9728.0
2402     johannes     johannes   9194.0
665        maatje       maatje   8014.0
412         janna        janna   7844.0
225       abraham      abraham   7621.0
1683        jacob        jacob   7208.0
4511         anna         anna   7127.0
1300      neeltje      neeltje   6708.0
6468    catharina    catharina   6210.0
3863       jacoba       jacoba   5637.0
2035     jannetje     jannetje   5267.0
1732     leendert     leendert   4254.0
2234         sara         sara   4148.0


In [8]:
lvl_3_first_name_variants['first_letter_same'] = lvl_3_first_name_variants.apply(lambda r: r[0][0] == r[1][0], axis=1)

In [10]:
lvl_3_first_name_variants[~lvl_3_first_name_variants['first_letter_same']].sort_values("count", ascending=False)

,name1,name2,count,first_letter_same
328,cornelis,kornelis,1480.0,False
2074,cornelia,kornelia,1026.0,False
5549,catharina,katharina,467.0,False
4480,helena,lena,248.0,False
1854,anna,maria,236.0,False
6053,tannetje,jannetje,161.0,False
4662,elena,helena,139.0,False
3533,janna,maria,117.0,False
1124,johanna,anna,87.0,False
5148,klazina,clazina,84.0,False


In [12]:

counts_per_name1 = lvl_3_first_name_variants[['name1', 'name2']].groupby(['name1']).count()
print counts_per_name1

             name2
name1             
'jan             1
a                1
aadriana         1
aafje            1
aagje            3
aagte            2
aagtje           6
aalbrecht        2
aalbregt         2
aaldert          1
aale             1
aaltie           1
aaltje           5
aaltjen          1
aana             1
aantje           1
aara             1
aarend           1
aarnhout         1
aarnoud          5
aarnoudt         2
aarnout          5
aaron            1
aart             1
aartje           2
abel             2
abeldina         1
aberdina         2
aberina          1
abig             2
...            ...
wouterus         1
woutrina         1
xaveria          1
xaverius         1
yda              2
yep              1
zacharias        1
zacharina        1
zacharius        1
zacherina        1
zacheus          2
zalihus          1
zantina          1
zara             1
zeeger           1
zeegert          1
zeger            3
zegerina         1
zegert           2
zelie       

In [13]:
name_variants_first_name = dict()
for index, row in lvl_3_first_name_variants.iterrows():
    if row['name1'] not in name_variants_first_name:
        name_variants_first_name[row['name1']] = []
    name_variants_first_name[row['name1']].append(row['name2'])


In [14]:
print name_variants_first_name['zeger']

['zeegert', 'zegert', 'zeger']


In [22]:
nv_counter_first_names = Counter()
for k in name_variants_first_name.keys():
    nv_counter_first_names[k] = len(name_variants_first_name[k])

In [34]:
pd.DataFrame.from_dict(nv_counter_first_names, orient='index').sort_values(0, ascending=False)

,0
catharina,27
jozina,19
janna,17
apolonia,15
jan,15
johannes,14
maatje,13
marinus,13
gerard,13
anna,13
